In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
color = sns.color_palette()
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#import plotly.tools as tls
#import plotly.express as px
import re
#import nltk
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
from os import path
#from PIL import Image
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import re

# Preprocessing
- For further data visualization some data needs to me added/modified. 
- The list of mentioned users gets extracted and every user mention is stored in a separate row
- The mentioned party gets updated
- The gender is of the MPs is missing, so the dataset gets merged with the MdB dataset

- Bei Control Group muss man aufpassen wegen Explode, weil beim Mergen mit den Twitter Usernames wahrscheinliche noch nicht alle Politiker in der Liste sind, deshalb so wening FDP z.B.

In [27]:
df = pd.read_csv("../full_datasets/control_group_mentions_predicted.csv")
#df = pd.read_csv("../full_datasets/mentions_predicted.csv")
#df = df.drop(['Unnamed: 0'], axis=1)
len(df)

/tmp/ipykernel_1114/1289300866.py:1: DtypeWarning: Columns (2,3,5,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../full_datasets/control_group_mentions_predicted.csv")


1534833

In [28]:
df = df.drop_duplicates()
len(df)

1534833

In [29]:
df.head(2)

,Unnamed: 0.1,index,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,quote_count,retweets,replies,likes,mentioned,Runs,Partei,model_predictions
0,0,0,693298,1412000720321134598,Delbertalexan10,33,3165,Biographie? Echt interessant...,NaN,@M_Richter65 @SMBAH05 @RTLde @kloeppelpeter @A...,2021-08-24 21:18:03+00:00,0.0,0.0,0.0,0.0,"['M_Richter65', 'SMBAH05', 'RTLde', 'kloeppelp...",Gruene_polis_1,Bündnis 90/Die Grünen,1
1,1,1,693299,1225125362763489281,Hsnbrg_eiee,18,770,Kritik der reinen Emotion,Stgt,@Super_Sastre @TinaHassel @ABaerbock Meinen Si...,2021-08-24 21:18:03+00:00,0.0,0.0,0.0,1.0,"['Super_Sastre', 'TinaHassel', 'ABaerbock']",Gruene_polis_1,Bündnis 90/Die Grünen,0


In [30]:
import ast
# Convert mentions to list
new = []
for index, row in df.iterrows():
    try:
        tolists = ast.literal_eval(row["mentioned"])
        new.append(tolists)
    except:
        #row["mentioned"] = []
        new.append("NA")

In [31]:
len(df)

1534833

In [35]:
df["mentioned_list"] = new

# Explode function to store every mention in a separate row
exploded_df = df.explode("mentioned_list")
exploded_df = exploded_df.reset_index()
print(len(exploded_df))
exploded_df.head(2)

4520554


,level_0,Unnamed: 0.1,index,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,...,created_at,quote_count,retweets,replies,likes,mentioned,Runs,Partei,model_predictions,mentioned_list
0,0,0,0,693298,1412000720321134598,Delbertalexan10,33,3165,Biographie? Echt interessant...,NaN,...,2021-08-24 21:18:03+00:00,0.0,0.0,0.0,0.0,"['M_Richter65', 'SMBAH05', 'RTLde', 'kloeppelp...",Gruene_polis_1,Bündnis 90/Die Grünen,1,M_Richter65
1,0,0,0,693298,1412000720321134598,Delbertalexan10,33,3165,Biographie? Echt interessant...,NaN,...,2021-08-24 21:18:03+00:00,0.0,0.0,0.0,0.0,"['M_Richter65', 'SMBAH05', 'RTLde', 'kloeppelp...",Gruene_polis_1,Bündnis 90/Die Grünen,1,SMBAH05


In [33]:
# Convert username to lower to be able to merge it
exploded_df['mentioned_list'] = exploded_df['mentioned_list'].map(lambda x: x.lower() if isinstance(x,str) else x)
exploded_df.head(2)

,level_0,Unnamed: 0.1,index,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,...,created_at,quote_count,retweets,replies,likes,mentioned,Runs,Partei,model_predictions,mentioned_list
0,0,0,0,693298,1412000720321134598,Delbertalexan10,33,3165,Biographie? Echt interessant...,NaN,...,2021-08-24 21:18:03+00:00,0.0,0.0,0.0,0.0,"['M_Richter65', 'SMBAH05', 'RTLde', 'kloeppelp...",Gruene_polis_1,Bündnis 90/Die Grünen,1,m_richter65
1,0,0,0,693298,1412000720321134598,Delbertalexan10,33,3165,Biographie? Echt interessant...,NaN,...,2021-08-24 21:18:03+00:00,0.0,0.0,0.0,0.0,"['M_Richter65', 'SMBAH05', 'RTLde', 'kloeppelp...",Gruene_polis_1,Bündnis 90/Die Grünen,1,smbah05


In [34]:
len(exploded_df)

4520554

## Merge exploded dataframe with extracted twitter usernames to get right party

In [15]:
twitter = pd.read_csv("twitter_usernames_extracted.csv",sep=";")
twitter['username'] = twitter['username'].map(lambda x: x.lower() if isinstance(x,str) else x)
twitter.head(2)

,Unnamed: 0.1,Unnamed: 0,full_name,party,twitter,username
0,0,0,Sanae Abdi,SPD,https://twitter.com/abdisanae'],abdisanae
1,1,1,Valentin Abel,FDP,['https://twitter.com/Valentin_C_Abel'],valentin_c_abel


In [13]:
exploded_df = exploded_df.drop(["level_0", "Unnamed: 0.1", "index", "Unnamed: 0"], axis=1)

In [19]:
test = exploded_df.merge(twitter, left_on='mentioned_list', right_on='username',suffixes=['', '_'])
#test = test.drop(["Unnamed: 0.1_", "Unnamed: 0_", "twitter", "level_0", "username_"],axis=1)
test.head(3)

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,quote_count,retweets,...,Runs,Partei,model_predictions,mentioned_list,Unnamed: 0.1,Unnamed: 0,full_name,party,twitter,username_
0,1164231742091386882,MatthiasBausch3,230,9233,"Pädagoge (Ambulante Hilfen), Mitglied der ak.m...",NaN,@SilkeGebel @KonstantinNotz @ABaerbock Deutsch...,2021-08-24 21:16:26+00:00,0.0,0.0,...,Gruene_polis_1,Bündnis 90/Die Grünen,0,konstantinnotz,474,474,Dr. Konstantin von Notz,Bündnis 90/Die Grünen,['http://twitter.com/KonstantinNotz'],konstantinnotz
1,959872400757284864,demokratienetz,2821,94774,"Online-Redaktion, Netzwerk, Spenden: https://t...","Baden-Württemberg, Deutschland",@SilkeGebel @KonstantinNotz @ABaerbock Der Bei...,2021-08-24 21:11:30+00:00,0.0,0.0,...,Gruene_polis_1,Bündnis 90/Die Grünen,0,konstantinnotz,474,474,Dr. Konstantin von Notz,Bündnis 90/Die Grünen,['http://twitter.com/KonstantinNotz'],konstantinnotz
2,1414108115188977666,Harzman6,28,1713,NaN,NaN,@KaiGehring @KonstantinNotz Bis 2005 hat man s...,2021-08-24 11:50:48+00:00,0.0,0.0,...,Gruene_polis_1,Bündnis 90/Die Grünen,0,konstantinnotz,474,474,Dr. Konstantin von Notz,Bündnis 90/Die Grünen,['http://twitter.com/KonstantinNotz'],konstantinnotz


In [17]:
test = test.drop_duplicates()

In [20]:
len(test)

1527602

In [21]:
len(exploded_df)

4520554

In [ ]:
#test.to_csv("mentions_predicted_exploded.csv")

## Merge exploded df with MdB dataset to get sex of MP

In [22]:
%%capture
!pip install openpyxl

In [23]:
mdb = pd.read_excel("MdB_20WP_Kontaktdaten.xlsx")
mdb.head()

,Name BT,Vorname BT,Titel,Adel,Praefix,BT_Zusatz,Geschlecht,NAME BT Zusammensetzung 1,NAME BT Zusammensetzung 2,Fraktion,...,BL,Anrede,Anschrift,Anschrift Straße,Anschrift PLZ,Anschrift Ort,INTERN,E-Mail,Vorwahl,Vermittlung
0,Abdi,Sanae,NaN,NaN,NaN,NaN,Frau,Sanae Abdi,"Abdi, Sanae",SPD,...,NW,Sehr geehrte Frau Abgeordnete,Deutscher Bundestag,Platz der Republik 1,11011,Berlin,im Hause,sanae.abdi@bundestag.de,+49 30 227,0
1,Abel,Valentin,NaN,NaN,NaN,NaN,Herrn,Valentin Abel,"Abel, Valentin",FDP,...,BW,Sehr geehrter Herr Abgeordneter,Deutscher Bundestag,Platz der Republik 1,11011,Berlin,im Hause,valentin.abel@bundestag.de,+49 30 227,0
2,Abraham,Knut,NaN,NaN,NaN,NaN,Herrn,Knut Abraham,"Abraham, Knut",CDU/CSU,...,BB,Sehr geehrter Herr Abgeordneter,Deutscher Bundestag,Platz der Republik 1,11011,Berlin,im Hause,knut.abraham@bundestag.de,+49 30 227,0
3,Adler,Katja,NaN,NaN,NaN,NaN,Frau,Katja Adler,"Adler, Katja",FDP,...,HE,Sehr geehrte Frau Abgeordnete,Deutscher Bundestag,Platz der Republik 1,11011,Berlin,im Hause,katja.adler@bundestag.de,+49 30 227,0
4,Aeffner,Stephanie,NaN,NaN,NaN,NaN,Frau,Stephanie Aeffner,"Aeffner, Stephanie",BÜ90/GR,...,BW,Sehr geehrte Frau Abgeordnete,Deutscher Bundestag,Platz der Republik 1,11011,Berlin,im Hause,stephanie.aeffner@bundestag.de,+49 30 227,0


In [24]:
sex = test.merge(mdb, left_on='full_name', right_on='NAME BT Zusammensetzung 1',suffixes=['', '_'])
sex = sex.drop(["Anrede", "Anschrift", "Anschrift Straße", "Anschrift PLZ", "Anschrift Ort", "INTERN", "E-Mail", "Vorwahl", "Vermittlung"],axis=1)
sex = sex.drop(["Praefix", "BT_Zusatz", "NAME BT Zusammensetzung 1", "NAME BT Zusammensetzung 2", "Fraktion", "Wahlkreis-Nr.", "Wahlkreisname/Bundesland", "BL", "Partei", "Name BT", "Vorname BT", "Titel", "Adel"],axis=1)

In [25]:
len(sex)

1508039

In [26]:
sex.to_csv("control_group_mentions_predicted_exploded.csv")